In [46]:
#!pip install node
#!pip install npm
#!pip install sodapy
from sodapy import Socrata
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import requests
import os
import datetime
import csv

from config import austin_key
from config import austin_app

In [47]:
app_token= austin_app

url = "https://data.austintexas.gov/resource/fdj4-gpfu.json?"
#crime_history = pd.DataFrame()
i=1
for i in range (1, 15):
    
    app_token= austin_app
    crime= "DEL CONTROLLED SUB/NARCOTIC"


    query_url = f"{url}$$app_token={app_token}&$limit=50000&$offset={50000*i}"
    response = requests.get(query_url).json()
    if i == 1:
        crime_history=pd.DataFrame(response)
        
    else:
        
        crime_history=crime_history.append(response)
crime_history.count()

In [ ]:
moon_df = pd.read_csv("moon-phase-calendar-landscape_copy.csv")
moon_df = pd.DataFrame(moon_df)
moon_df = moon_df.drop(columns=['Unnamed: 3', 'Unnamed: 4'])
moon_df.head()
#change to date time object and rename column
moon_df['Local Date']= pd.to_datetime(moon_df['Local Date'])

moon_df.head()

In [ ]:
crime_df = pd.DataFrame(crime_history)
crime_df.head()
#if you want to group your data by time you have to do resample not .groupby(), can resample by days or by months, check time example pushed out last week
#compare if crime occurs more at night consistently vs increasing during full moon
#could sample by moon phase or day date 
#pick 10 full moon days and compare count to 10 non full moon days 

In [ ]:
crime_clean_df = crime_df[["crime_type", "occ_date_time", "occ_date", "occ_time", "rep_date"]].copy()
#checking if occurence is actualy date/time
#crime_clean_df.dtypes
#convert to datetime object 
#pd.to_datetime(crime_clean_df['occ_date_time'])
#put back into dataframe overwrite column
crime_clean_df['occ_date_time']= pd.to_datetime(crime_clean_df['occ_date_time'])
crime_clean_df['occ_date']= pd.to_datetime(crime_clean_df['occ_date'])

crime_clean_df.head()

In [ ]:
#whatever one is typed first is the left df

crime_moon_df= crime_clean_df.merge(moon_df, left_on='occ_date', right_on='Local Date', how='left')
crime_moon_df.sample(50)

In [ ]:
crime_moon_df['Lunar Phase'].value_counts()

In [ ]:
#create pivot table of the crime duplicates and sort to find top 50 reported crimes 
duplicates = crime_clean_df.pivot_table(index = 'crime_type', aggfunc= 'size')
#data series can sort index or sort values, default to sort by index so we want to sort by the values instead
duplicates.sort_values(ascending= False).head(50)

In [ ]:
#replace one crime type with the more generic types ex "bicycle theft" to "theft", grouping like crimes in the top 50
#df["column1"].replace({"a": "x", "b": "y"}, inplace=True)
crime_grouped = crime_moon_df.replace({"THEFT BY SHOPLIFTING" : "THEFT", "AUTO THEFT" : "THEFT", "THEFT OF BICYCLE" : "THEFT", "THEFT OF SERVICE":"THEFT", "THEFT OF LICENSE PLATE": "THEFT",
                                       "THEFT FROM AUTO":"THEFT", "THEFT FROM PERSON":"THEFT", "THEFT FROM BUILDING":"THEFT", "THEFT OF TRAILER" :"THEFT",
                                       "DWI":"INTOXICATION CRIME", "PUBLIC INTOXICATION":"INTOXICATION CRIME", "DWI 2ND": "INTOXICATION CRIME", 
                                        "DWI  .15 BAC OR ABOVE": "INTOXICATION CRIME", "DRIVING WHILE INTOX / FELONY" : "INTOXICATION CRIME",
                                       "POSSESSION OF MARIJUANA": "DRUG RELATED", "POSS CONTROLLED SUB/NARCOTIC":"DRUG RELATED", "POSS OF DRUG PARAPHERNALIA":"DRUG RELATED",
                                       "ASSAULT WITH INJURY":"ASSAULT", "ASSAULT W/INJURY-FAM/DATE VIOL" : "ASSAULT", "ASSAULT BY CONTACT" :"ASSAULT", "ASSAULT BY THREAT": "ASSAULT",
                                       "ASSAULT BY CONTACT FAM/DATING": "ASSAULT", "AGG ASSAULT":"ASSAULT", "AGG ASLT STRANGLE/SUFFOCATE":"ASSAULT",
                                       "AGG ASSAULT FAM/DATE VIOLENCE": "ASSAULT", "FELONY ENHANCEMENT/ASSLT W/INJ" : "ASSAULT", "ASSAULT  CONTACT-SEXUAL NATURE" :"ASSAULT",
                                       "AGG ROBBERY/DEADLY WEAPON": "ROBBERY", "ROBBERY BY ASSAULT" :"ROBBERY",
                                       "BURGLARY OF VEHICLE":"BURGLARY", "BURGLARY OF SHED/DETACHED GARAGE/STORAGE UNIT" : "BURGLARY", "BURGLARY OF RESIDENCE" : "BURGLARY", "BURGLARY NON RESIDENCE" :"BURGLARY",
                                       "FRAUD - OTHER" :"FRAUD", "IDENTITY THEFT":"FRAUD", "FORGERY AND PASSING":"FRAUD", "CRED CARD ABUSE - OTHER":"FRAUD", "DEBIT CARD ABUSE":"FRAUD",
                                       "FAMILY DISTURBANCE":"DISTURBANCE","DATING DISTURBANCE":"DISTURBANCE","FAMILY DISTURBANCE/PARENTAL":"DISTURBANCE","DISTURBANCE - OTHER":"DISTURBANCE"})

crime_grouped.head()

In [ ]:
#repeat checking duplicates to qc for typos and missed categories that can be grouped 

duplicate_groups = crime_grouped.pivot_table(index = 'crime_type', aggfunc= 'size')
duplicate_groups.sort_values(ascending= False).head(50)

#cut the small database at runaway child 

#Plots by year

In [ ]:
#use groupby occurence date to get a count of the full number of reports submitted by date 
date_df= crime_moon_df.groupby(['occ_date']).count()
date_df.reset_index(inplace=True)
#drop first row of data becuase it is incomplete due to how it was pulled from the database 
date_df = date_df.iloc[1:]
date_df.head()

In [ ]:
#line graph of date vs count for all years
plt.plot(date_df['occ_date'], date_df['crime_type'])

plt.show()

There is a lot of noise on the data set and we want to see the x axis in months
Going to break up the data by year in order to visualize it on a more appropriate time scale 

In [ ]:
#make a data frame for every year using .loc
fourteen_df = date_df.loc[(date_df['occ_date'] >= "2014-04-15") & (date_df['occ_date'] <= "2014-12-31")]
#use .loc to select only full moons in order to make the full moon pics on the plot
fourteen_full_moon = moon_df.loc[(moon_df['Lunar Phase'] == 'Full Moon') & (moon_df['Local Date'] >= "2014-04-15") & (moon_df['Local Date'] <= "2014-12-31")]
fourteen_full_moon

In [ ]:
#use list comprehendion to create coordinates for the vertical line
#need to use a for loop in order to pull all of the full moon dates for the year 
#x value will be the date and y value will be successive points along the range of the y axis 
#specify color 

plt.figure(figsize= (10,5))
plt.plot(fourteen_df['occ_date'], fourteen_df['crime_type'])
#line where x value stays the same and y value changes 
#x for x in range 200 to 450
for i, row in fourteen_full_moon.iterrows():
    x= [row['Local Date'] for x in range (200, 450)]
    y= [x for x in range (200, 450)]
    
    plt.plot(x, y, color="red")

plt.title("Correlation of Full Moons and Crime Reports per Day- 2014")
plt.xlabel("Months")
plt.ylabel("Reports per Day")
plt.legend(labels = ['Report Count', 'Full Moon'])
plt.show()

In [ ]:
#repeat for 2015
fifteen_df = date_df.loc[(date_df['occ_date'] >= "2015-01-01") & (date_df['occ_date'] <= "2015-12-31")]

fifteen_full_moon = moon_df.loc[(moon_df['Lunar Phase'] == 'Full Moon') & (moon_df['Local Date'] >= "2015-01-01") & (moon_df['Local Date'] <= "2015-12-31")]

plt.figure(figsize= (10,5))
plt.plot(fifteen_df['occ_date'], fifteen_df['crime_type'])
#line where x value stays the same and y value changes 
#x for x in range 200 to 450
for i, row in fifteen_full_moon.iterrows():
    x= [row['Local Date'] for x in range (200, 450)]
    y= [x for x in range (200, 450)]
    
    plt.plot(x, y, color="red")

plt.title("Correlation of Full Moons and Crime Reports per Day- 2015")
plt.xlabel("Months")
plt.ylabel("Reports per Day")
plt.legend(labels = ['Report Count', 'Full Moon'])
plt.show()

In [ ]:
#repeat for 2016
sixteen_df = date_df.loc[(date_df['occ_date'] >= "2016-01-01") & (date_df['occ_date'] <= "2016-12-31")]
sixteen_full_moon = moon_df.loc[(moon_df['Lunar Phase'] == 'Full Moon') & (moon_df['Local Date'] >= "2016-01-01") & (moon_df['Local Date'] <= "2016-12-31")]

plt.figure(figsize= (10,5))
plt.plot(sixteen_df['occ_date'], sixteen_df['crime_type'])
#line where x value stays the same and y value changes 
#x for x in range 200 to 450
for i, row in sixteen_full_moon.iterrows():
    x= [row['Local Date'] for x in range (200, 450)]
    y= [x for x in range (200, 450)]
    
    plt.plot(x, y, color="red")

plt.title("Correlation of Full Moons and Crime Reports per Day- 2016")
plt.xlabel("Months")
plt.ylabel("Reports per Day")
plt.legend(labels = ['Report Count', 'Full Moon'])
plt.show()

In [ ]:
#repeat for 2017
seventeen_df = date_df.loc[(date_df['occ_date'] >= "2017-01-01") & (date_df['occ_date'] <= "2017-12-31")]

seventeen_full_moon = moon_df.loc[(moon_df['Lunar Phase'] == 'Full Moon') & (moon_df['Local Date'] >= "2017-01-01") & (moon_df['Local Date'] <= "2017-12-31")]

plt.figure(figsize= (10,5))
plt.plot(seventeen_df['occ_date'], seventeen_df['crime_type'])
#line where x value stays the same and y value changes 
#x for x in range 200 to 450
for i, row in seventeen_full_moon.iterrows():
    x= [row['Local Date'] for x in range (200, 450)]
    y= [x for x in range (200, 450)]
    
    plt.plot(x, y, color="red")

plt.title("Correlation of Full Moons and Crime Reports per Day- 2017")
plt.xlabel("Months")
plt.ylabel("Reports per Day")
plt.legend(labels = ['Report Count', 'Full Moon'])
plt.show()

In [ ]:
#repeat for 2018
eighteen_df = date_df.loc[(date_df['occ_date'] >= "2018-01-01") & (date_df['occ_date'] <= "2018-12-31")]

eighteen_full_moon = moon_df.loc[(moon_df['Lunar Phase'] == 'Full Moon') & (moon_df['Local Date'] >= "2018-01-01") & (moon_df['Local Date'] <= "2018-12-31")]

plt.figure(figsize= (10,5))
plt.plot(eighteen_df['occ_date'], eighteen_df['crime_type'])
#line where x value stays the same and y value changes 
#x for x in range 200 to 450
for i, row in eighteen_full_moon.iterrows():
    x= [row['Local Date'] for x in range (200, 450)]
    y= [x for x in range (200, 450)]
    
    plt.plot(x, y, color="red")

plt.title("Correlation of Full Moons and Crime Reports per Day- 2018")
plt.xlabel("Months")
plt.ylabel("Reports per Day")
plt.legend(labels = ['Report Count', 'Full Moon'])
plt.show()

In [ ]:
#repeat for 2019
nineteen_df = date_df.loc[(date_df['occ_date'] >= "2019-01-01") & (date_df['occ_date'] <= "2019-12-31")]

nineteen_full_moon = moon_df.loc[(moon_df['Lunar Phase'] == 'Full Moon') & (moon_df['Local Date'] >= "2019-01-01") & (moon_df['Local Date'] <= "2019-12-31")]

plt.figure(figsize= (10,5))
plt.plot(nineteen_df['occ_date'], nineteen_df['crime_type'])
#line where x value stays the same and y value changes 
#x for x in range 200 to 450
for i, row in nineteen_full_moon.iterrows():
    x= [row['Local Date'] for x in range (200, 450)]
    y= [x for x in range (200, 450)]
    
    plt.plot(x, y, color="red")

plt.title("Correlation of Full Moons and Crime Reports per Day- 2019")
plt.xlabel("Months")
plt.ylabel("Reports per Day")
plt.legend(labels = ['Report Count', 'Full Moon'])
plt.show()

In [ ]:
#repeat for 2020
twenty_df = date_df.loc[(date_df['occ_date'] >= "2020-01-01") & (date_df['occ_date'] <= "2020-12-31")]

twenty_full_moon = moon_df.loc[(moon_df['Lunar Phase'] == 'Full Moon') & (moon_df['Local Date'] >= "2020-01-01") & (moon_df['Local Date'] <= "2020-12-31")]

plt.figure(figsize= (10,5))
plt.plot(twenty_df['occ_date'], twenty_df['crime_type'])
#line where x value stays the same and y value changes 
#x for x in range 200 to 450
for i, row in twenty_full_moon.iterrows():
    x= [row['Local Date'] for x in range (200, 450)]
    y= [x for x in range (200, 450)]
    
    plt.plot(x, y, color="red")

plt.title("Correlation of Full Moons and Crime Reports per Day- 2020")
plt.xlabel("Months")
plt.ylabel("Reports per Day")
plt.legend(labels = ['Report Count', 'Full Moon'])
plt.show()

In [ ]:
#create a df with only full moon phases across the full 7 year period (can be used for box and whisker plots) 
full_moon_list = ['Full Moon']

full_moon_df = crime_grouped.loc[crime_grouped['Lunar Phase'].isin(full_moon_list)]

full_moon_df.sample(10)

In [ ]:
#create df with only new moon phases across full 7 year period (can be used for box and whisker plots)
new_moon_list = ['New Moon']

new_moon_df = crime_grouped.loc[crime_grouped['Lunar Phase'].isin(new_moon_list)]

new_moon_df.sample(10)

In [ ]:
#create df with only first quarter phases across full 7 year period (can be used for box and whisker plots)
new_moon_list = ['First Quarter']

first_quarter_df = crime_grouped.loc[crime_grouped['Lunar Phase'].isin(new_moon_list)]

first_quarter_df.sample(10)

In [ ]:
#create df with only last quarter moon phases across full 7 year period (can be used for box and whisker plots)
new_moon_list = ['Last Quarter']

last_quarter_df = crime_grouped.loc[crime_grouped['Lunar Phase'].isin(new_moon_list)]

last_quarter_df.sample(10)

In [ ]:
#create a list of the top 10 most reported crimes                
crime_list = ['THEFT', 'DISTURBANCE', 'BURGLARY', 'ASSAULT', 'INTOXICATION CRIME', 'CRIMINAL MISCHIEF', 'DRUG RELATED', 'FRAUD', 'HARASSMENT', 'RUNAWAY CHILD']

top_crime_df = crime_grouped.loc[crime_grouped['crime_type'].isin(crime_list)]

top_crime_df.sample(10)

In [ ]:
top_crime_df['crime_type'].value_counts()

In [ ]:
top_crime_df['Lunar Phase'].value_counts()

In [ ]:
# combine the full moon and new moon data frames
lunar_one_df = pd.merge(full_moon_df, new_moon_df, on=['crime_type', 'occ_date_time', 'occ_date', 'occ_time', 'rep_date', 'Lunar Phase', 'Local Date', 'Local Time'], how='left').fillna(0)
lunar_one_df.head()

In [ ]:
# Combine the first quarter and last quarter data frames
lunar_two_df = pd.merge(first_quarter_df, last_quarter_df, on=['crime_type', 'occ_date_time', 'occ_date', 'occ_time', 'rep_date', 'Lunar Phase', 'Local Date', 'Local Time'], how='left').fillna(0)
lunar_two_df.head()

In [ ]:
# combine all of the lunar phase data frames
combined_lunar_phase_df = pd.merge(lunar_one_df, lunar_two_df, on=['crime_type', 'occ_date_time', 'occ_date', 'occ_time', 'rep_date', 'Lunar Phase', 'Local Date', 'Local Time'], how='left').fillna(0)
combined_lunar_phase_df.head()

In [ ]:
combined_lunar_phase_df['Lunar Phase'].value_counts()

In [ ]:
combined_lunar_phase_df.count()

In [ ]:
combined_lunar_phase_df.dtypes

In [ ]:

intox_list = ['INTOXICATION CRIME']

intox_crime_df = crime_grouped.loc[crime_grouped['crime_type'].isin(intox_list)]

intox_crime_df.sample(10)

In [ ]:
intox_crime_df['crime_type'].value_counts()

In [ ]:
intox_crime_df['Lunar Phase'].value_counts()

In [ ]:
assault_list = ['ASSAULT']

assault_df = crime_grouped.loc[crime_grouped['crime_type'].isin(assault_list)]

assault_df.sample(10)

In [ ]:
assault_df['crime_type'].value_counts()

In [ ]:
assault_df['Lunar Phase'].value_counts()

In [ ]:
steal_list = ['THEFT', 'BURGLARY']

steal_df = crime_grouped.loc[crime_grouped['crime_type'].isin(steal_list)]

steal_df.sample(10)

In [ ]:
steal_df['crime_type'].value_counts()

In [ ]:
steal_df['Lunar Phase'].value_counts()

In [ ]:
drug_list = ['DRUG RELATED']

drug_df = crime_grouped.loc[crime_grouped['crime_type'].isin(drug_list)]

drug_df.sample(10)

In [ ]:
drug_df['crime_type'].value_counts()

In [ ]:
drug_df['Lunar Phase'].value_counts()

-COMPLETED annual daily report numbers with full moon lines plotted on top
    -2014
    -2015
    -2016
    -2017
    -2018
    -2019
    -2020
    -2021 (erin)
    
-COMPLETED plot report numbers over all reporting years

-box and whisker plots for all years of data, separate boxes for separate moon phases to see if there is a trend (jon)

-line graph of 3-5 months with all moon phase lines plotted on top (Fatima)

-measure of central tendency for all full moons in datasets (report counts) (greg)

-split out violent vs nonviolent crimes for line graphs over 1-2 years and box and whisker plots 

-compare curves of only top 10 crimes with all crimes 
